In [12]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from config import db_password
from sqlalchemy import create_engine
import psycopg2
import io
import time

In [13]:
contributions_df = pd.read_csv('Totals_by_State.csv', low_memory=False)
contributions_df

,Rank,state,2020_Total_Contributions,2020_Democrats,2020_Republicans,2018_Total_Contributions,2018_Democrats,2018_Republicans,2016_Total_Contributions,2016_Democrats,2016_Republicans
0,30,Alabama,"$72,681,601",26.35%,71.33%,"$24,417,482",29.83%,64.16%,"$21,340,249",25.90%,72.27%
1,47,Alaska,"$20,469,701",35.44%,45.44%,"$4,819,621",49.57%,48.04%,"$6,078,471",41.71%,53.91%
2,19,Arizona,"$159,737,336",46.15%,52.44%,"$50,550,487",42.80%,54.59%,"$52,686,535",41.70%,56.13%
3,31,Arkansas,"$69,797,741",32.79%,65.31%,"$25,956,720",28.72%,69.06%,"$49,490,346",35.51%,62.84%
4,1,California,"$1,671,408,159",69.21%,28.52%,"$616,260,806",67.47%,27.79%,"$756,675,466",68.54%,29.21%
5,14,Colorado,"$196,381,141",62.53%,35.90%,"$52,281,612",54.24%,41.55%,"$73,892,527",57.39%,41.19%
6,20,Connecticut,"$141,963,644",70.59%,27.45%,"$52,931,450",64.12%,32.55%,"$91,105,154",60.39%,36.98%
7,24,Delaware,"$99,319,912",71.84%,27.19%,"$5,799,262",59.49%,38.42%,"$8,606,355",60.40%,37.96%
8,3,District of Columbia,"$810,335,383",66.84%,31.66%,"$511,923,092",57.06%,41.36%,"$481,157,263",56.83%,42.17%
9,5,Florida,"$649,564,630",36.77%,61.24%,"$214,498,755",42.48%,54.01%,"$317,694,017",40.81%,56.57%


In [14]:
margin_df = pd.read_csv('swing_state_margin.csv', low_memory=False)
margin_df

,state,2020,2016,2012,2020_results,2016_results,2012_results
0,Arizona,0.31,3.55,9.06,blue,red,red
1,Florida,3.36,1.20,0.88,red,red,blue
2,Georgia,0.24,5.13,7.82,blue,red,red
3,Michigan,2.78,2.96,9.50,blue,blue,blue
4,Minnesota,7.11,0.23,7.69,blue,red,blue
5,Nevada,2.39,2.10,6.68,blue,blue,blue
6,New Hampshire,7.35,2.42,5.58,blue,blue,blue
7,North Carolina,1.35,0.37,2.04,red,blue,red
8,Pennsylvania,1.16,3.66,5.39,blue,red,blue
9,Texas,5.58,0.72,4.45,red,red,red


In [15]:
merged_inner = pd.merge(left=contributions_df, right=margin_df, left_on='state', right_on='state')
merged_inner

,Rank,state,2020_Total_Contributions,2020_Democrats,2020_Republicans,2018_Total_Contributions,2018_Democrats,2018_Republicans,2016_Total_Contributions,2016_Democrats,2016_Republicans,2020,2016,2012,2020_results,2016_results,2012_results
0,19,Arizona,"$159,737,336",46.15%,52.44%,"$50,550,487",42.80%,54.59%,"$52,686,535",41.70%,56.13%,0.31,3.55,9.06,blue,red,red
1,5,Florida,"$649,564,630",36.77%,61.24%,"$214,498,755",42.48%,54.01%,"$317,694,017",40.81%,56.57%,3.36,1.20,0.88,red,red,blue
2,12,Georgia,"$256,912,683",38.42%,60.09%,"$61,402,643",33.61%,63.22%,"$77,341,241",33.87%,63.89%,0.24,5.13,7.82,blue,red,red
3,13,Michigan,"$200,930,159",49.43%,48.99%,"$75,563,731",49.04%,48.67%,"$85,637,741",46.45%,52.01%,2.78,2.96,9.50,blue,blue,blue
4,21,Minnesota,"$126,874,458",62.45%,36.12%,"$50,402,024",55.37%,42.56%,"$47,749,526",53.17%,44.36%,7.11,0.23,7.69,blue,red,blue
5,9,Nevada,"$325,602,382",31.01%,67.33%,"$158,727,113",31.85%,65.35%,"$126,728,749",30.24%,67.35%,2.39,2.10,6.68,blue,blue,blue
6,40,New Hampshire,"$36,431,819",70.73%,27.77%,"$9,064,179",66.15%,31.21%,"$14,387,923",54.70%,44.04%,7.35,2.42,5.58,blue,blue,blue
7,18,North Carolina,"$174,574,133",49.68%,48.74%,"$52,514,773",44.05%,53.13%,"$65,900,012",43.76%,54.16%,1.35,0.37,2.04,red,blue,red
8,11,Pennsylvania,"$265,835,612",60.00%,38.14%,"$105,666,359",56.80%,40.15%,"$160,384,290",65.13%,33.62%,1.16,3.66,5.39,blue,red,blue
9,4,Texas,"$690,959,847",34.77%,63.27%,"$295,940,533",40.00%,56.33%,"$323,124,854",30.96%,66.61%,5.58,0.72,4.45,red,red,red


In [19]:
merged_left = pd.merge(left=contributions_df, right=margin_df, how='left', left_on='state', right_on='state')
merged_left

,Rank,state,2020_Total_Contributions,2020_Democrats,2020_Republicans,2018_Total_Contributions,2018_Democrats,2018_Republicans,2016_Total_Contributions,2016_Democrats,2016_Republicans,2020,2016,2012,2020_results,2016_results,2012_results
0,30,Alabama,"$72,681,601",26.35%,71.33%,"$24,417,482",29.83%,64.16%,"$21,340,249",25.90%,72.27%,NaN,NaN,NaN,NaN,NaN,NaN
1,47,Alaska,"$20,469,701",35.44%,45.44%,"$4,819,621",49.57%,48.04%,"$6,078,471",41.71%,53.91%,NaN,NaN,NaN,NaN,NaN,NaN
2,19,Arizona,"$159,737,336",46.15%,52.44%,"$50,550,487",42.80%,54.59%,"$52,686,535",41.70%,56.13%,0.31,3.55,9.06,blue,red,red
3,31,Arkansas,"$69,797,741",32.79%,65.31%,"$25,956,720",28.72%,69.06%,"$49,490,346",35.51%,62.84%,NaN,NaN,NaN,NaN,NaN,NaN
4,1,California,"$1,671,408,159",69.21%,28.52%,"$616,260,806",67.47%,27.79%,"$756,675,466",68.54%,29.21%,NaN,NaN,NaN,NaN,NaN,NaN
5,14,Colorado,"$196,381,141",62.53%,35.90%,"$52,281,612",54.24%,41.55%,"$73,892,527",57.39%,41.19%,NaN,NaN,NaN,NaN,NaN,NaN
6,20,Connecticut,"$141,963,644",70.59%,27.45%,"$52,931,450",64.12%,32.55%,"$91,105,154",60.39%,36.98%,NaN,NaN,NaN,NaN,NaN,NaN
7,24,Delaware,"$99,319,912",71.84%,27.19%,"$5,799,262",59.49%,38.42%,"$8,606,355",60.40%,37.96%,NaN,NaN,NaN,NaN,NaN,NaN
8,3,District of Columbia,"$810,335,383",66.84%,31.66%,"$511,923,092",57.06%,41.36%,"$481,157,263",56.83%,42.17%,NaN,NaN,NaN,NaN,NaN,NaN
9,5,Florida,"$649,564,630",36.77%,61.24%,"$214,498,755",42.48%,54.01%,"$317,694,017",40.81%,56.57%,3.36,1.20,0.88,red,red,blue


In [17]:
merged_inner.to_csv('swing_state.csv')

In [20]:
merged_left.to_csv('swing_state_dataset.csv')